<a href="https://colab.research.google.com/github/sushirito/Molecular-Dynamics/blob/OpenMM/tleap_simplification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install AmberTools
!pip install -q condacolab
import condacolab
condacolab.install()  # Installs Conda
!mamba install -c conda-forge ambertools -y

✨🍰✨ Everything looks OK!

Looking for: ['ambertools']

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache

Pinned packages:
  - python 3.11.*
  - python 3.11.*
  - python_abi 3.11.* *cp311*
  - cuda-version 12.*


Transaction

  Prefix: /usr/local

  All requested packages already installed



In [2]:
!pip install openmm
!pip install MDAnalysis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 69.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 85.7 MB/s eta 0:00:00


In [5]:
# Prepare the tleap input file
tleap_script = """
source leaprc.protein.ff14SB
source leaprc.water.tip3p
loadAmberParams frcmod.ions234lm_126_tip3p

# Create an empty unit named 'solv'
solv = createUnit solv

# Fill 'solv' with TIP3P water (10.0 Å buffer)
solvateBox solv TIP3PBOX 10.0

# Add one Hg2+ and one Cl-
addIons2 solv Hg2+ 1
addIons2 solv Cl- 1

# Save Amber topology and coordinate files
saveAmberParm solv solv.prmtop solv.inpcrd
quit
"""


# Save tleap script
with open('tleap.in', 'w') as f:
    f.write(tleap_script)

# Run tleap
!tleap -f tleap.in

# Generate trajectory using OpenMM
from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, PDBReporter, PME, HBonds
from openmm import LangevinMiddleIntegrator, Platform
from openmm.unit import kelvin, picoseconds, nanometer

# Load Amber files
prmtop = AmberPrmtopFile('solv.prmtop')
inpcrd = AmberInpcrdFile('solv.inpcrd')

# Create system
system = prmtop.createSystem(nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picoseconds, 0.004*picoseconds)
platform = Platform.getPlatformByName('CPU')

# Initialize simulation
simulation = Simulation(prmtop.topology, system, integrator, platform)
simulation.context.setPositions(inpcrd.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('hgtrajectory.pdb', 100))
simulation.step(5000)

print("Simulation completed. Trajectory saved as 'hgtrajectory.pdb'.")


-I: Adding /usr/local/dat/leap/prep to search path.
-I: Adding /usr/local/dat/leap/lib to search path.
-I: Adding /usr/local/dat/leap/parm to search path.
-I: Adding /usr/local/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /usr/local/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /usr/local/dat/leap/cmd/leaprc.protein.ff14SB done
Log file: ./leap.log
Loading parameters: /usr/local/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /usr/local/dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading title:
ff14SB protein backbone and sidechain parameters
Loading library: /usr/local/dat/leap/lib/amino12.lib
Loading library: /usr/local/dat/leap/lib/aminoct12.lib
Loading library: /usr/local/dat/leap/lib/aminont12.lib
----- Source: /usr/local/dat/leap/cmd/leaprc.water.tip3p
----- Source of /usr/local/dat/leap/cm

FileNotFoundError: [Errno 2] No such file or directory: 'solv.prmtop'

In [23]:
import subprocess
from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, PDBReporter, PME, HBonds
from openmm import LangevinMiddleIntegrator, Platform
from openmm.unit import kelvin, picoseconds, nanometer

# Create a minimal structure to solvate
dummy_pdb = """\
ATOM      1  O   HOH A   1       0.000   0.000   0.000  1.00  0.00
END
"""
with open("dummy.pdb", "w") as f:
    f.write(dummy_pdb)

# Corrected tleap input script
tleap_script = """\
source leaprc.protein.ff14SB
source leaprc.water.tip3p
loadAmberParams frcmod.ions234lm_126_tip3p

mol = loadPdb dummy.pdb
solv = solvateBox mol TIP3PBOX 10.0

addIons2 solv Hg2+ 1
addIons2 solv Cl-  1

saveAmberParm solv solv.prmtop solv.inpcrd
quit
"""
with open('tleap.in', 'w') as f:
    f.write(tleap_script)

# Run tleap and capture errors
try:
    result = subprocess.run(['tleap', '-f', 'tleap.in'], check=True, capture_output=True, text=True)
except subprocess.CalledProcessError as e:
    print("TLeap failed:")
    print(e.stderr)
    raise

# Continue with simulation if tleap succeeded
prmtop = AmberPrmtopFile('solv.prmtop')
inpcrd = AmberInpcrdFile('solv.inpcrd')

system = prmtop.createSystem(nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picoseconds, 0.004*picoseconds)
platform = Platform.getPlatformByName('CPU')

simulation = Simulation(prmtop.topology, system, integrator, platform)
simulation.context.setPositions(inpcrd.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('hgtrajectory.pdb', 100))
simulation.step(5000)

print("Simulation completed. Trajectory saved as 'hgtrajectory.pdb'.")


TLeap failed:



CalledProcessError: Command '['tleap', '-f', 'tleap.in']' returned non-zero exit status 21.